# Malignant Comment Analysis
***

Social media has been trending for the past decades. New forms of media, such as Instagram, Reddit, Discord, online forums, and chat applications, have become widely used as means of communication. However, there have been many cases of cyberbullying, hate speeches, trolls, and inappropriate languages used within these applications that have to be moderated. In this notebook, we will go through the steps of building an ML model using Naives Bayes Classifier to identify texts that are innaproppriate.

We will be doing two models:
1. Naives Bayes from scratch
2. Naives Bayes with sklearn.naives_bayes

The dataset used is from kaggle: https://www.kaggle.com/datasets/surekharamireddy/malignant-comment-classification

Github link to this notepad:

***
### Imports

In [41]:
from math import exp,log
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
import re

***
### Exploratory Data Analysis ###

<b> Let's read the Data from the dataset <b>
    

In [24]:
totalDf = pd.read_csv('MalignantComment.csv')
totalDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 8 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   id                159571 non-null  object
 1   comment_text      159571 non-null  object
 2   malignant         159571 non-null  int64 
 3   highly_malignant  159571 non-null  int64 
 4   rude              159571 non-null  int64 
 5   threat            159571 non-null  int64 
 6   abuse             159571 non-null  int64 
 7   loathe            159571 non-null  int64 
dtypes: int64(6), object(2)
memory usage: 9.7+ MB


In [25]:
totalDf.head()

,id,comment_text,malignant,highly_malignant,rude,threat,abuse,loathe
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


**Note** There are six different classifications from which a comment can be ruled under. We can create a new column in this dataframe which encompasses every other classification. We will call the column 'negative' which will imply that the text was classified any of the six different classifications.

In [26]:
#Create a new list that holds the values of the new column 'negative'
list_negative = []

#iterate through the dataframe and append '1' if the text was classified inappropriate, else append '0'
for i,row in totalDf.iterrows():
    if(row['malignant'] == 1 or row['highly_malignant'] == 1 or row['rude'] == 1 or row['threat'] == 1 or row['abuse'] == 1 or row['loathe'] == 1):
        list_negative.append(1)
    else:
        list_negative.append(0)
        
#Insert the list into the dataframe
totalDf.insert(loc=8,column =  "negative", value = list_negative)

In [27]:
totalDf.head()

,id,comment_text,malignant,highly_malignant,rude,threat,abuse,loathe,negative
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,0


**Note** Next, we need to preprocess the data in column 'comment_text'. As of now, each entry is one giant string that includes unwanted special characters. We will use regex to place characters outside of the english alphabet and numbers, replacing it with a empty space. We will then use the lowercase of all strings.

* From scratch implementation will need a list of clean words.
* Sklearn implementation will just need the cleaned text string.

In [28]:
#List for scratch implementation column
list_comment_text = []
#List for sklearn implementation column
sk_comment_text = []

#Iterate over the dataframe to replace, make lower case, and split/append accordingly
for i,row in totalDf.iterrows():
    long_text = re.sub('[^0-9a-zA-Z]+', ' ', row['comment_text']).lower()
    list_comment_text.append(long_text.split())
    sk_comment_text.append(long_text)

#Create columns using the lists created
totalDf.insert(loc=1,column =  "list_comment_text", value = list_comment_text)
totalDf.insert(loc=1,column =  "sk_comment_text", value = sk_comment_text)

In [29]:
totalDf.head()

,id,sk_comment_text,list_comment_text,comment_text,malignant,highly_malignant,rude,threat,abuse,loathe,negative
0,0000997932d777bf,explanation why the edits made under my userna...,"[explanation, why, the, edits, made, under, my...",Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,0
1,000103f0d9cfb60f,d aww he matches this background colour i m se...,"[d, aww, he, matches, this, background, colour...",D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,0
2,000113f07ec002fd,hey man i m really not trying to edit war it s...,"[hey, man, i, m, really, not, trying, to, edit...","Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,0
3,0001b41b1c6bb37e,more i can t make any real suggestions on imp...,"[more, i, can, t, make, any, real, suggestions...","""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,0
4,0001d958c54c6e35,you sir are my hero any chance you remember wh...,"[you, sir, are, my, hero, any, chance, you, re...","You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,0


Now our data is clean!

***
## Naives Bayes Classifier from scratch ###

The Naives Bayes Classifier is an ML technique derived from Bayes Theorem. Bayes Theorem States:

**P(A | B) * P(B) = P(B | A) * P(A)**<br>
or<br>
**P(A | B) = P(B | A) * P(A) / P(B)**<br>

We can use Bayes theorem to vision our problem case as:
***
#### *P(Inappropriate | Words in List) = P(Inappropriate) * P(Words in List | Inappropriate) / P(Words in List)*
#### *P(Appropriate | Words in List) = P(Appropriate) * P(Words in List | Appropriate) / P(Words in List)*
***
&emsp;***P(Inappropriate | Words in List)*** is our predicted probability that given text is inappropriate

&emsp;***P(Appropriate | Words in List)*** is our predicted probability that given text is appropriate

&emsp;***P(Inappropriate)*** is the probability that a generalized text is inappropriate<br>
&emsp;&emsp; = (# of texts label 1)/(# of total texts)

&emsp;***P(Appropriate)*** is the probability that a generalized text is appropriate<br>
&emsp;&emsp; = (# of texts label 0)/(# of total texts)

&emsp;***P(Words in List)*** is the probability of a specific generalized text<br>
&emsp;&emsp; This probability is very hard to compute, and will not be using this probability as we compare<br>
&emsp;&emsp; P(Appropriate | Words in List) with P(Inappropriate | Words in List)

&emsp;***P(Words in List | Inappropriate)*** is the probability that the words in the text are in an inappropriate labeled text<br>
***


Let's dive separately into ***P(Words in List | Inappropriate)***

Since Naives Bayes assumes independence for each word in the text: Therefor <br>
&emsp;&emsp;*P(Words in List | Inappropriate)* = <br>
&emsp;&emsp;*P(Word1 and Word2 and Word3 ... WordN | Inappropriate)* = <br>
&emsp;&emsp;*P(Word1| Inappropriate) * P(Word2| Inappropriate) * ... * P(WordN| Inappropriate)*<br>
&emsp;&emsp;**Where**<br>
&emsp;&emsp;*P(WordN| Inappropriate)* = (# of WordN appearances in Inappropriate text)/(# of total words in Inappropriate text)

However, there may be cases where WordN doesn't appear in our training Inappropriate text
To account for this, let us add 1 to all unique values in all text, giving us:<br>
***P(WordN | Inappropriate)* = <br>
((# of WordN appearances in Inappropriate text) + 1)/((# of total words in Inappropriate text) + (# of total unique words in all text)**
***

Let's go into the implementation so see how this is coded

In [30]:
class naivesBayesSentiment:
    
    def __init__(self):

        #Calculate the number of rows in training set
        self.n_rows = 0
        
        #Calculate P(Malignant), P(Highly Malignant), ...
        self.prob_neg = 0
        
        #Calculate P(Word | Malignant) and P(Word | not Malignant)
        self.neg_Dict = {}
        self.pos_Dict = {}
    
    
    def _prepopulateWordDict(self, wordList):
        returnDict = {}
        wordSet = set(wordList)
        for word in wordSet:
            returnDict[word] = 1
        return returnDict
    
    def _calculate_Neg_Dict(self, x_train, y_train):
        neg_list = []
        pos_list = []
        for x,y in zip(x_train, y_train):
            if(y == 1):
                neg_list.extend(x)
            else:
                pos_list.extend(x)

        #prepopulate the dictionary with smoothing curve alpha
        total_Word_List = neg_list + pos_list
        neg_Dict = self._prepopulateWordDict(total_Word_List)
        pos_Dict = self._prepopulateWordDict(total_Word_List)
        
        #create a dictionary of
        #key -> word
        #value -> word count + 1
        for word in neg_list:
            if (word in neg_Dict):
                neg_Dict[word] += 1
        
        for word in pos_list:
            if (word in pos_Dict):
                pos_Dict[word] += 1
                
        #Sort the Dictionaries
        neg_Dict= dict(sorted(neg_Dict.items(), key=lambda item: item[1], reverse = True)) 
        pos_Dict= dict(sorted(pos_Dict.items(), key=lambda item: item[1], reverse = True)) 
        return neg_Dict, pos_Dict
    
    def fit(self, x_train, y_train):
        self.n_rows = len(x_train)
        self.prob_neg = sum(y_train)/self.n_rows
        self.neg_Dict, self.pos_Dict = self._calculate_Neg_Dict(x_train, y_train)
        return self
    
    def predict(self, X):
        #X will be a 2-d array
        pred_y = []
        prob_neg = 1
        prob_pos = 1
        sum_Neg = sum(self.neg_Dict.values())
        sum_Pos = sum(self.pos_Dict.values())
        for x_test in X:
            prob_neg = self.prob_neg
            prob_pos = 1 - self.prob_neg
            for word in x_test:
                if(word in self.neg_Dict):
                    #formula for naivebayes
                    prob_neg *= self.neg_Dict[word]/sum_Neg
                    prob_pos *= self.pos_Dict[word]/sum_Pos
            if(prob_neg > prob_pos):
                pred_y.append(1)
            else:
                pred_y.append(0)
        return pred_y
    
        
        
    

Split the data into training and testing sets

In [31]:
X = totalDf['list_comment_text'].tolist()
Y = totalDf['negative'].tolist()
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=.20)

**Create the model and fit the training data**


In [32]:
nbs_Model = naivesBayesSentiment().fit(X_train,y_train)

**Calling the fit(X,y) function will generate two dictionaries**
* Dictionary of unique word counts in Appropriate
* Dictionary of unique word counts in Inappropriate

**Note** These dictionaries have been adjusted so that all unique values in all texts have a initial value of 1.<br>
e.g Dictionary['key'] == ((# of WordN appearances in Inappropriate text) + 1)<br>
e.g sum of all values in Dictionary == ((# of total words in Inappropriate text) + (# of total unique words in all text)<br>

Let's print out some of the dictionary values

In [40]:
print(nbs_Model.neg_Dict)

{'you': 33578, 'i': 20970, 'the': 18917, 'a': 17990, 'and': 14318, 'to': 14215, 'is': 11141, 'of': 10402, 'fuck': 8602, 'your': 7755, 'that': 7408, 'it': 7405, 'are': 7103, 'in': 6100, 'my': 5075, 't': 5045, 'this': 4895, 'me': 4763, 'not': 4440, 'on': 4335, 'for': 4183, 'have': 4121, 's': 3972, 'do': 3611, 'with': 3582, 'be': 3253, 'all': 3167, 'what': 3115, 'like': 3083, 'wikipedia': 3043, 'an': 2923, 'so': 2859, 'as': 2823, 'suck': 2801, 'can': 2780, 'ass': 2725, 'am': 2722, 'u': 2684, 'go': 2531, 'will': 2511, 'fucking': 2497, 'hate': 2469, 'if': 2451, 'was': 2274, 'just': 2230, 'don': 2230, 'but': 2176, 'up': 2174, 'he': 2165, 'or': 2114, 'who': 2080, 're': 2067, 'gay': 2055, 'no': 2048, 'shit': 2041, 'get': 2037, 'about': 2030, 'faggot': 1936, 'know': 1903, 'page': 1891, 'off': 1855, 'die': 1836, 'yourself': 1814, 'by': 1758, 'because': 1756, 'from': 1745, 'fat': 1738, 'people': 1684, 'nigger': 1682, 'they': 1629, 'why': 1616, 'bitch': 1508, 'm': 1445, 'one': 1372, 'at': 1367, 'j

**Calling the predict(X) function will apply the Bayes Theorem formula**

&emsp;&emsp;***P(Appropriate | Words in List) = P(Appropriate) * P(Words in List | Appropriate) / P(Words in List)***

In [33]:
pred_y = nbs_Model.predict(X_test)

### Prediction Analysis for NBC ###



Some helper functions to calculate precision and recall scores

In [34]:
def calculate_precision(y_true, y_pred, pos_label_value=1.0):

    if(len(y_true) != len(y_pred)):
        return -1
    
    TrueP = 0 
    FalseP = 0
    #We need to calculate total True positives and False postives
    for yt, yp in zip(y_true,y_pred):
        if(yt == pos_label_value and yp == pos_label_value):
            #add to TrueP
            TrueP += 1
        if(yt != pos_label_value and yp == pos_label_value):
            #add to FalseP
            FalseP += 1
    
    precision = TrueP/(TrueP + FalseP)
    
    return precision

def calculate_recall(y_true, y_pred, pos_label_value=1.0):
    
    TrueP = 0 
    FalseN = 0
    #We need to calculate total True positives and False postives
    for yt, yp in zip(y_true,y_pred):
        if(yt == pos_label_value and yp == pos_label_value):
            #add to TrueP
            TrueP += 1
        if(yt == pos_label_value and yp != pos_label_value):
            #add to FalseP
            FalseN += 1
    
    
    recall = TrueP/(TrueP + FalseN)
    # your code here
    
    
    return recall

Calculation scores

In [43]:

ac = accuracy_score(pred_y, y_test)
print('Accuracy score: ' + str(ac))
print('Precision score: ' + str(calculate_precision(y_test, pred_y)))
print('Recall score: ' + str(calculate_recall(y_test, pred_y)))

Accuracy score: 0.9440075199749334
Precision score: 0.7714505579068872
Recall score: 0.6269543464665416


This model has an relatively decent accuracy score. This score is high in comparison to the precision and recall scores since the score of true negatives are high.

Ways to improve this model include:
* Improving the lexigraph of what is stored in the dictionary. There are infinite amount of ways to mistype, abbreviate, and create new words. There may be a way to categorize these situations better.
* Increasing training data

Unfortunately, since this is a Naives Bayes, there will always be a limitation that each word in the testing text will be independent of each other. That is not the case, since the gramatical structure of the english language is somewhat dependent on the text phrase before hand.

## Bonus: Naives Bayes Classifier using sklearn ###

In [45]:
X_sklearn = totalDf['sk_comment_text']
Y_sklearn = totalDf['negative']

X_train_sk, X_test_sk, y_train_sk, y_test_sk = train_test_split(X_sklearn, Y_sklearn, test_size=0.2)

In [47]:
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer


pipe = Pipeline(steps=[('vectorize', CountVectorizer(ngram_range=(1, 1), token_pattern=r'\b\w+\b')),
                       ('classifier', MultinomialNB())])
pipe.fit(X_train_sk, y_train_sk)
y_predict = pipe.predict(X_test_sk)

print('Accuracy score: ' + str(accuracy_score(y_test_sk, y_predict)))
print('Precision score: ' + str(calculate_precision(y_test_sk, y_predict)))
print('Recall score: ' + str(calculate_recall(y_test_sk, y_predict)))


Accuracy score: 0.9422528591571362
Precision score: 0.7492732558139535
Recall score: 0.6413685847589424
